# Caribbean Island Habitat

This indicator was used in 2023 Southeast Blueprint.

Created by Rua Mordecai

In [31]:
import os, string
import arcpy
# Use pandas since this indicator relies on a bunch of data manupulation from excel
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [32]:
# spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
OutputWorkspace = r"D:\Blueprint\2023\finalIndicatorEdits\CaribbeanBlueprint2023_FinalIndicators\CaribbeanBlueprint2023_FinalIndicators\SpatialData"

In [33]:
# raster used for cell size, extent, and snapping
CaribbeanRaster = r"D:\Blueprint\2023\extent\VIPR_Extent_v6.tif"

In [34]:
# island boundary data
islandBoundary = r"D:\Blueprint\2023\extent\Islands3Size.tif"

In [35]:
# critical habitat input data
criticalHabitatData = r"D:\Blueprint\2022\Indicators\Islands\crithab_all_layers\CRITHAB_POLY.shp"

In [36]:
# threatened and invasive species input data
invasiveAndThreatenedData = "D:/Blueprint/2023/Caribbean/Indicators/Islands/ThreatenedIslandBiodiversityDatabase/CaribbeanData_TIBDataRequest_29Nov2022.xlsx"

In [37]:
# location of GAP species data to provide more specifics for species on larger islands
prGAP = r"D:\Blueprint\2023\Caribbean\Indicators\Islands\SpeciesFromGAP\PR"
viGAP = r"D:\Blueprint\2023\Caribbean\Indicators\Islands\SpeciesFromGAP\VI"

In [38]:
indicatorName = "CaribbeanIslandHabitat.tif"

In [39]:
# Sometimes arcpro is fussy about not overwriting things
arcpy.env.overwriteOutput = True

## Prep threatened and invasive species data

### Prep Puerto Rico gap data

In [10]:
# change to data workspace
arcpy.env.workspace = prGAP

In [11]:
# make a list of indicators to be clipped
RList = arcpy.ListRasters()

In [12]:
print(RList)

['agelaius_xanthomus_habitat.img', 'amazona_vittata_habitat.img', 'anolis_roosevelti_habitat.img', 'caprimulgus_noctitherus_habitat.img', 'caprimulgus_noctitherus_habitatFixed.img', 'eleutherodactylus_eneidae_habitat.img', 'eleutherodactylus_gryllus_habitat.img', 'eleutherodactylus_hedricki_habitat.img', 'eleutherodactylus_jasperi_habitat.img', 'eleutherodactylus_locustus_habitat.img', 'eleutherodactylus_portoricensis_habitat.img', 'eleutherodactylus_portoricensis_habitatFixed.img', 'eleutherodactylus_richmondi_habitat.img', 'eleutherodactylus_wightmanae_habitat.img', 'epicrates_monensis_granti_habitat.img', 'prSpCount.tif', 'spCount.tif', 'spCountClean.tif', 'sphaerodactylus_levinsi_habitat.img']


In [13]:
# figure out how many species are included.DONT USE MORE THAN 26 or it'll cause weird issues
prSpNum=len(RList)

In [14]:
# Deal with screwy species layers. Format for this layer is 0/1 not 0/1/2
if "eleutherodactylus_portoricensis_habitat.img" in RList:
    out_raster = arcpy.sa.Plus("eleutherodactylus_portoricensis_habitat.img", 1); out_raster.save("eleutherodactylus_portoricensis_habitatFixed.img")
    # replace old version with fixed version in raster list
    RList=list(map(lambda x: x.replace('eleutherodactylus_portoricensis_habitat.img','eleutherodactylus_portoricensis_habitatFixed.img'),RList))       

In [15]:
# Deal with screwy species layers. This this layer has a weird noData square in the middle of the island
if "caprimulgus_noctitherus_habitat.img" in RList:
    # change raster so weird noData square is 0 which is conistent with how noData is coded in other layers
    out_raster = arcpy.sa.Con(arcpy.sa.IsNull("caprimulgus_noctitherus_habitat.img"),0,"caprimulgus_noctitherus_habitat.img"); out_raster.save("caprimulgus_noctitherus_habitatFixed.img")
    # replace old version with fixed version in raster list
    RList=list(map(lambda x: x.replace('caprimulgus_noctitherus_habitat.img','caprimulgus_noctitherus_habitatFixed.img'),RList))       

In [16]:
print(RList)

['agelaius_xanthomus_habitat.img', 'amazona_vittata_habitat.img', 'anolis_roosevelti_habitat.img', 'caprimulgus_noctitherus_habitatFixed.img', 'caprimulgus_noctitherus_habitatFixed.img', 'eleutherodactylus_eneidae_habitat.img', 'eleutherodactylus_gryllus_habitat.img', 'eleutherodactylus_hedricki_habitat.img', 'eleutherodactylus_jasperi_habitat.img', 'eleutherodactylus_locustus_habitat.img', 'eleutherodactylus_portoricensis_habitatFixed.img', 'eleutherodactylus_portoricensis_habitatFixed.img', 'eleutherodactylus_richmondi_habitat.img', 'eleutherodactylus_wightmanae_habitat.img', 'epicrates_monensis_granti_habitat.img', 'prSpCount.tif', 'spCount.tif', 'spCountClean.tif', 'sphaerodactylus_levinsi_habitat.img']


In [17]:
# make a consecutive list of letters based on number of rasters
prGAPLetters = list(string.ascii_lowercase)[:len(RList)]

In [18]:
# make a formula using those letters
prGAPFormula = ''
for i in prGAPLetters:
    prGAPFormula += i + " + "
# remove the last + from the formula
prGAPFormula = prGAPFormula[:-3]    

In [19]:
#Sum rasters in raster calculator
output_raster = arcpy.sa.RasterCalculator(RList,prGAPLetters,prGAPFormula); output_raster.save("sumSpecies.tif")

In [20]:
# Subtract the total number of species analyzed from the summed raster. This gets to the total number 
# of overlapping species per pixel b/c Puerto Rico gap has 1 for absense and 2 for presence
out_raster = arcpy.sa.Minus("sumSpecies.tif", prSpNum); out_raster.save("spCount.tif")

In [21]:
# Deal with zero values in inconsitently modeled coastal water and edges
out_raster = arcpy.sa.Con("spCount.tif", 0, "spCount.tif", "Value < 0"); out_raster.save("spCountClean.tif")

In [22]:
# Snap, convert noData to zero, and expand extent based on Caribbean Blueprint
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
    out_raster = arcpy.sa.Con(arcpy.sa.IsNull("spCountClean.tif"),0,"spCountClean.tif"); out_raster.save("prSpCount.tif")

### Prep VI GAP data

In [10]:
# change to data workspace
arcpy.env.workspace = viGAP

In [11]:
# make a list of indicators to be clipped
RList = arcpy.ListRasters()

In [12]:
print(RList)

['Eleutherodactylus_lentus_PHM.img', 'Epicrates_monensis_granti_PHM.img', 'viSpCount.tif', 'VIspCountClean.tif']


In [13]:
# figure out how many species are included.DONT USE MORE THAN 26 or it'll cause weird issues
viSpNum=len(RList)

In [14]:
# make a consecutive list of letters based on number of rasters
viGAPLetters = list(string.ascii_lowercase)[:len(RList)]

In [15]:
# make a formula using those letters
viGAPFormula = ''
for i in viGAPLetters:
    viGAPFormula += i + " + "
# remove the last + from the formula
viGAPFormula = viGAPFormula[:-3]    

In [16]:
#Sum rasters in raster calculator
output_raster = arcpy.sa.RasterCalculator(RList,viGAPLetters,viGAPFormula); output_raster.save("VIsumSpecies.tif")

In [17]:
# Subtract the total number of species analyzed from the summed raster. This gets to the total number 
# of overlapping species per pixel b/c Virgin Islands gap has 1 for absense and 2 for presence
out_raster = arcpy.sa.Minus("VIsumSpecies.tif", viSpNum); out_raster.save("VIspCount.tif")

In [18]:
# Deal with zero values used as noData
out_raster = arcpy.sa.Con("VIspCount.tif", 0, "VIspCount.tif", "Value < 0"); out_raster.save("VIspCountClean.tif")

In [19]:
# Snap, convert noData to zero, and expand extent based on Caribbean Blueprint
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
    out_raster = arcpy.sa.Con(arcpy.sa.IsNull("VIspCountClean.tif"),0,"VIspCountClean.tif"); out_raster.save("viSpCount.tif")

### Combine Puerto Rico and Virgin Islands data

In [40]:
# Change the workspace to where I am saving the outputs
arcpy.env.workspace = OutputWorkspace

In [21]:
# Sum the layers
out_raster = arcpy.sa.Plus(viGAP+r"\VIspCount.tif", prGAP+r"\prspCount.tif"); out_raster.save("GAPspCount.tif")

In [22]:
# Convert values to indicator bins
out_raster = arcpy.sa.Con("GAPspCount.tif", 2, "GAPspCount.tif", "Value > 1"); out_raster.save("GAPspBins.tif")

In [23]:
# Multiply the layers by 10 to make them easier to combine with other data
out_raster = arcpy.sa.Times("GAPspBins.tif", 10); out_raster.save("GAPspBins10.tif")

### Prep invasive data

In [24]:
# read invasive data into pandas data frame
invasiveSpeciesData = pd.read_excel(invasiveAndThreatenedData,sheet_name="Invasive Species on Islands",usecols="A,B,C,E,F,G,H,J")

In [25]:
# Reduce data to only islands covered in 2023 Blueprint
invasiveSpeciesDataVIPR = invasiveSpeciesData.query('Region_Archipelago in ["Greater Antilles (Puerto Rican Islands)", "US Virgin Islands","US Virgin Islands (St. Croix Islands)","US Virgin Islands (St. Thomas Islands)"]')

In [26]:
# remove species records called 'NONE'. These are placeholders when an island doesn't have invasives
invasiveSpeciesDataVIPR = invasiveSpeciesDataVIPR.query('Common_Name != "NONE"')

In [27]:
# Each invasive species is a separate row. Get number of invasives by counting the rows for each island. 
# There are replacate island IDs so use the code instead of island name
invasiveCounts = pd.pivot_table(invasiveSpeciesDataVIPR, index=['Island_GID_Code'], aggfunc= {
    'Invasive_Status': 'count', 'Corrected_Area_KM2':'mean'})

### Prep threatened species data

In [28]:
# read threatened species data into pandas data frame
threatenedSpeciesData = pd.read_excel(invasiveAndThreatenedData,sheet_name="Threatened Species on Islands",usecols="F,G,H,J,K")

In [29]:
# Reduce data to only islands covered in 2023 Blueprint
threatenedSpeciesDataVIPR = threatenedSpeciesData.query('Region_Archipelago in ["Greater Antilles (Puerto Rican Islands)", "US Virgin Islands","US Virgin Islands (St. Croix Islands)","US Virgin Islands (St. Thomas Islands)"]')

In [30]:
# Include only records with current confirmed or potential breeding.
# While info on exterpated species could be helpful in other places, all the ones in the 2023 Blueprint areas are on the large 
# islands >50km2 which get removed from this particular part of the analysis
threatenedSpeciesDataVIPR = threatenedSpeciesDataVIPR.query('Present_Breeding_Status in ["Confirmed","Potential Breeding"]')

In [31]:
# Each threatened species is a separate row. Get number of threatened species by counting the rows for each island
threatenedSpeciesCounts = pd.pivot_table(threatenedSpeciesDataVIPR, index=['Island_GID_Code'], aggfunc= {
    'Present_Breeding_Status': 'count'})

### Combine threatened and invasive data

In [32]:
# Merge invasive and threatened species data. Since the data doesn't have zeros, use the Island_GID_Code field and
# fill in missing data with 0
speciesMerge = pd.merge(invasiveCounts, threatenedSpeciesCounts, how="outer", left_index=True, right_index=True)
speciesMergeFill = speciesMerge.fillna(0)


In [33]:
# include only smaller islands (<50 km2). This removes islands roughly the size of St. Thomas and larger. That includes Mona, Vieques, St. Croix, St. John, and Puerto Rico
speciesMergeSmallerIslands = speciesMergeFill.query('Corrected_Area_KM2 < 50')

In [34]:
# read island info data
islandInfoData = pd.read_excel(invasiveAndThreatenedData,sheet_name="Invasive Species on Islands",usecols="A,B,C,E,F,G")

In [35]:
# Reduce data to only islands covered in 2023 Blueprint
islandInfoDataVIPR = islandInfoData.query('Region_Archipelago in ["Greater Antilles (Puerto Rican Islands)", "US Virgin Islands","US Virgin Islands (St. Croix Islands)","US Virgin Islands (St. Thomas Islands)"]')

In [36]:
# include only smaller islands (<50 km2). This removes islands roughly the size of St. Thomas and larger. That includes Mona, Vieques, St. Croix, St. John, and Puerto Rico
islandInfoSmallerIslands = islandInfoDataVIPR.query('Corrected_Area_KM2 < 50')

In [37]:
# get the latitude and Longitude for each island
islandInfo = pd.pivot_table(islandInfoSmallerIslands, index=['Island_GID_Code'], aggfunc= {
    'Corrected_Latitude': 'mean', 'Corrected_Longitude': 'mean'})

In [38]:
# Combine species data and island lat/long
islandSpeciesRaw = pd.merge(speciesMergeSmallerIslands, islandInfo, how="outer", left_index=True, right_index=True)

In [39]:
# Drop island size column and fill in no data in species columns with 0
islandSpecies4column = islandSpeciesRaw.drop(['Corrected_Area_KM2'],axis = 1)

In [40]:
islandSpecies = islandSpecies4column.fillna(0)

In [41]:
# Add a new column that calculates the value for the indicator
islandSpecies.loc[(islandSpecies['Invasive_Status'] == 0) & (islandSpecies['Present_Breeding_Status'] > 1), 'ValueForIndicator'] = 6
islandSpecies.loc[(islandSpecies['Invasive_Status'] == 0) & (islandSpecies['Present_Breeding_Status'] == 1), 'ValueForIndicator'] = 5
islandSpecies.loc[(islandSpecies['Invasive_Status'] == 0) & (islandSpecies['Present_Breeding_Status'] == 0), 'ValueForIndicator'] = 4
islandSpecies.loc[(islandSpecies['Invasive_Status'] > 0) & (islandSpecies['Present_Breeding_Status'] > 1), 'ValueForIndicator'] = 3  
islandSpecies.loc[(islandSpecies['Invasive_Status'] > 0) & (islandSpecies['Present_Breeding_Status'] == 1), 'ValueForIndicator'] = 2 
islandSpecies.loc[(islandSpecies['Invasive_Status'] > 0) & (islandSpecies['Present_Breeding_Status'] == 0), 'ValueForIndicator'] = 1 

## Start spatial analysis

In [10]:
# Change the workspace to where I am saving the outputs
arcpy.env.workspace = OutputWorkspace

In [43]:
# Save species data as csv to use in ArcPro
islandSpecies.to_csv(OutputWorkspace+"\islandSpecies.csv")


In [44]:
arcpy.management.XYTableToPoint(OutputWorkspace+"\islandSpecies.csv", "islandSpecies.shp",
                                "Corrected_Longitude", "Corrected_Latitude")

<Result 'D:\\Blueprint\\2023\\Caribbean\\Indicators\\Islands\\working\\islandSpecies.shp'>

In [11]:
# prep islands boundary raster. Reclass so all islands have same value
out_raster = arcpy.sa.Reclassify(islandBoundary, "Value", "1 NODATA;2 4 1", "DATA"); out_raster.save("islandBoundaryBin.tif")

In [12]:
# convert island raster to shapefile
arcpy.conversion.RasterToPolygon("islandBoundaryBin.tif", "islandBoundary.shp", "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)

<Result 'D:\\Blueprint\\2023\\Caribbean\\Indicators\\Islands\\working\\islandBoundary.shp'>

In [13]:
# Join island boundaries and island species data
arcpy.analysis.SpatialJoin("islandBoundary.shp", "islandSpecies.shp", "IslandsAndSpeciesJoin.shp")

<Result 'D:\\Blueprint\\2023\\Caribbean\\Indicators\\Islands\\working\\IslandsAndSpeciesJoin.shp'>

In [14]:
# Remove specific species from critical habitat due to issues they cause. So far that only includes yellow-shouldered
# Blackbird due to the large area of lower quality habitat it's critical habitat includes.
out_raster = arcpy.management.SelectLayerByAttribute(criticalHabitatData, "NEW_SELECTION", "comname <> 'Yellow-shouldered blackbird'", None)

In [15]:
# Export the shapefile with the species removed
out_raster2 = arcpy.conversion.FeatureClassToFeatureClass(out_raster, OutputWorkspace, "CriticalHabitatReduced.shp")

In [16]:
# Clip critical habitat to islands
arcpy.analysis.Clip("CriticalHabitatReduced.shp", "islandBoundary.shp", "CriticalHabitatIslands.shp", None)

<Result 'D:\\Blueprint\\2023\\Caribbean\\Indicators\\Islands\\working\\CriticalHabitatIslands.shp'>

In [17]:
# add a field island critical habitat to convert to raster, give a value of 100
arcpy.management.CalculateField("CriticalHabitatIslands.shp", "raster", "100", "PYTHON3", '', "SHORT")

<Result 'D:\\Blueprint\\2023\\Caribbean\\Indicators\\Islands\\working\\CriticalHabitatIslands.shp'>

In [18]:
# convert critical habitat to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
    arcpy.conversion.FeatureToRaster(in_features="CriticalHabitatIslands.shp", field="raster", out_raster="CriticalHabitatIslandsNull.tif", cell_size=CaribbeanRaster)

In [19]:
# change critical habitat raster so there isn't noData on islands
out_raster = arcpy.sa.Con(arcpy.sa.IsNull("CriticalHabitatIslandsNull.tif"),0,"CriticalHabitatIslandsNull.tif"); out_raster.save("CriticalHabitatIslands.tif")

In [20]:
# convert islands to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
    arcpy.conversion.FeatureToRaster(in_features="IslandsAndSpeciesJoin.shp", field="ValueForIn", out_raster="SpeciesIslands.tif", cell_size=CaribbeanRaster)

In [21]:
# Combine rasters for indicator categories
output_raster = arcpy.ia.RasterCalculator(["CriticalHabitatIslands.tif", "SpeciesIslands.tif","GAPspBins10.tif"],['a','b','c'],'a + b + c'); output_raster.save("AllDataCombined.tif")

In [22]:
# Step 1 to convert to final indicator values. Convert all critical habitat to 7s
out_raster = arcpy.sa.Con("AllDataCombined.tif", 7, "AllDataCombined.tif", "Value >= 100"); out_raster.save("AllDataCombinedCrit7.tif")

In [23]:
# Step 2 to convert to final indicator values. Reclass the rest of the values.
out_raster = arcpy.sa.Reclassify('AllDataCombinedCrit7.tif', "Value", "NODATA 0;0 1;1 1;2 2;3 3;4 4;5 5;6 6;7 7;10 2;11 2;12 2;13 3;14 4;15 5;16 6;20 3;23 3", "DATA"); out_raster.save("indicatorBin.tif")

In [24]:
# export nonclipped version of indicator
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
#    arcpy.management.CopyRaster("indicatorBin.tif", indicatorName, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [25]:
# clip to Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
    out_raster = arcpy.sa.ExtractByMask("indicatorBin.tif", CaribbeanRaster); out_raster.save(indicatorName)

## Finalize indicator

In [26]:
# set code block for next step
codeblock = """
def Reclass(v):
	if v == 7:
		return '7 = Island area with critical habitat for a threatened or endangered species'
	elif v == 6:
		return '6 = Island area with no invasive animals and 2+ imperiled species'
	elif v == 5:
		return '5 = Island area with no invasive animals and 1 imperiled species'
	elif v == 4:
		return '4 = Island area with no invasive animals'
	elif v == 3:
		return '3 = Island area with invasive animals and 2+ imperiled species'
	elif v == 2:
		return '2 = Island area with invasive animals and 1 imperiled species'
	elif v == 1:
		return '1 = Island area with invasive animals'
	elif v == 0:
		return '0 = Not an island'
"""

In [27]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(indicatorName, "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\Blueprint\\2023\\Caribbean\\Indicators\\Islands\\working\\CaribbeanIslandHabitat.tif'>

In [47]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 7:
		return 107
	if Value == 6:
		return 151
	if Value == 5:
		return 196
	if Value == 4:
		return 242
	if Value == 3:
		return 247
	if Value == 2:
		return 252
	if Value == 1:
		return 255
	else:
		return 255
		
def Reclass2(Value):
	if Value == 7:
		return 6
	if Value == 6:
		return 57
	if Value == 5:
		return 111
	if Value == 4:
		return 167
	if Value == 3:
		return 195
	if Value == 2:
		return 224
	if Value == 1:
		return 255
	else:
		return 255
		
def Reclass3(Value):
	if Value == 7:
		return 1
	if Value == 6:
		return 13
	if Value == 5:
		return 28
	if Value == 4:
		return 46
	if Value == 3:
		return 72
	if Value == 2:
		return 99
	if Value == 1:
		return 128
	else:
		return 255
"""

In [48]:
# calculate Red field
arcpy.management.CalculateField(indicatorName, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(indicatorName, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(indicatorName, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\Blueprint\\2023\\finalIndicatorEdits\\CaribbeanBlueprint2023_FinalIndicators\\CaribbeanBlueprint2023_FinalIndicators\\SpatialData\\CaribbeanIslandHabitat.tif'>